<a href="https://colab.research.google.com/github/Rifthi-tech/ai-recommendation-project-pcp/blob/main/AI_Based_Personalized_Product_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IMPORT PACKAGES**

# **LOAD DATASET FROM GOOGLE DRIVE**

# **PREPARE DATA FOR TRAINING**

# **CREATE INTERACTION MATRIX**

# **TRAIN KNN MODEL**

# **CREATE RECOMMENDATION FUNCTION**

# **ACCURACY TESTING**

# **SAVE MODEL & EXPORT**